In [1]:
import pandas as pd
import numpy as np
import os
import glob
import pickle
import contextlib

from tensorflow import keras as k
from tensorflow.keras.layers import *
from funciones import *

import ipywidgets as widgets
from ipywidgets import VBox
import io
from PIL import Image


tamaño=450

In [2]:
import tensorflow as tf
from tensorflow import keras as k
from tensorflow.keras.layers import *
from funciones import *

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
df_nodulos=pd.read_csv("nodulos_descripcion.csv",header=0,index_col=0)
df_nodulos.columns=["Forma","Margen","Orientación","Ecogenicidad","Característica Posterior","Halo Ecogénico", 'Sugestivo', 'Forma2',
       'Margen2', 'Orientación2', 'Ecogenicidad2', 'Característica Posterior2',
       'Halo Ecogénico2', 'Sugestivo2', 'BIRADS', 'Resultados', 'Num Nódulos']

#Creamos el vocabulario que compone nuestra lista de descriptores que obtenemos de entre las opciones de la plantilla excel
Formas=[l for l in list(set(list(df_nodulos["Forma"])+list(df_nodulos["Forma2"]))) if l==l]
Margenes=[l for l in list(set(list(df_nodulos["Margen"])+list(df_nodulos["Margen2"]))) if l==l]
Orientaciones=["paralela","antiparalela"]
Ecogenicidades=[l for l in list(set(list(df_nodulos["Ecogenicidad"])+list(df_nodulos["Ecogenicidad2"]))) if l==l]
Posteriores=["sin cambios","refuerzo","sombra","mixto"]
Halos=["sí","no"]
Sugestividades=[l for l in list(set(list(df_nodulos["Sugestivo"])+list(df_nodulos["Sugestivo2"]))) if l==l]
BIRADS=["2","3","4A","4B","4C","5"]
Resultados=["benign","malignant"]
#Tipo=["quiste","fibroadenoma","carcinoma ductal invasivo","carcinoma ductal in situ","linfoma"]
#voc=set(Formas+Margenes+Orientaciones+Ecogenicidades+Posteriores+Halos+Sugestividades+BIRADS+Resultados+Tipo)
voc=set(Formas+Margenes+Orientaciones+Ecogenicidades+Posteriores+Halos+Sugestividades+BIRADS+Resultados)
voc.add("\t")
voc.add("\n")
voc=list(voc)
voc.sort()

In [6]:
word_to_idx = dict((word, idx) for idx, word in enumerate(voc))
idx_to_word = dict((idx, word) for idx, word in enumerate(voc))
descripciones_x={}
descripciones_y={}

#Vamos a ver escribir las características en una sola lista por cada nódulo eliminando los valores nan.
nodulos_x=[]
nodulos_y=[]

for ind, row in df_nodulos.iterrows():
    nodulo_x=["\t"]
    nodulo_y=[]
    for carac in row[:7]:
        if pd.isna(carac):
            continue
        else:
            nodulo_x.append(str(carac))
            nodulo_y.append(str(carac))
    for carac in row[14:16]:
        if pd.isna(carac):
            continue
        else:
            nodulo_x.append(str(carac))
            nodulo_y.append(str(carac))
    nodulo_y.append("\n")
    nodulos_x.append(nodulo_x)
    nodulos_y.append(nodulo_y)
    
    if row["Num Nódulos"]==2:
        nodulo_x=["\t"]
        nodulo_y=[]
        for carac in row[7:16]:
            if pd.isna(carac):
                continue
            else:
                nodulo_x.append(str(carac))
                nodulo_y.append(str(carac))
        nodulo_y.append("\n")
        nodulos_x.append(nodulo_x)
        nodulos_y.append(nodulo_y)
    
    
max_len_x=max([len(nodulo) for nodulo in nodulos_x])
i=0
for ind, row in df_nodulos.iterrows():
    input_data = np.zeros((max_len_x, len(voc)), dtype='float32')
    for j in range(len(nodulos_x[i])):
        input_data[j,word_to_idx[nodulos_x[i][j]]]=1
    descripciones_x[ind]=[input_data]
    if row["Num Nódulos"]==2:
        input_data = np.zeros((max_len_x, len(voc)), dtype='float32')
        i=i+1
        for j in range(len(nodulos_x[i])):
            input_data[j,word_to_idx[nodulos_x[i][j]]]=1
        descripciones_x[ind].append(input_data)
    i=i+1
        
i=0
max_len_y=max([len(nodulo) for nodulo in nodulos_y])
for ind, row in df_nodulos.iterrows():
    input_data = np.zeros((max_len_y, len(voc)), dtype='float32')
    for j in range(len(nodulos_y[i])):
        input_data[j,word_to_idx[nodulos_y[i][j]]]=1
    descripciones_y[ind]=[input_data]
    if row["Num Nódulos"]==2:
        input_data = np.zeros((max_len_y, len(voc)), dtype='float32')
        i=i+1
        for j in range(len(nodulos_y[i])):
            input_data[j,word_to_idx[nodulos_y[i][j]]]=1
        descripciones_y[ind].append(input_data)
    i=i+1

In [7]:
#Solo si no se entrena antes
EncoderLSTM = k.models.load_model('EncoderYOLOLSTM.h5')
decoder=LSTM_Attention()
word_embedding=Dense(len(voc),activation="softmax")


encoder_model_inf = k.Model(inputs=EncoderLSTM.inputs, outputs=EncoderLSTM.outputs)

# Create decoder input states for inference
decoder_state_input_h = Input(shape=(256))
decoder_state_input_c = Input(shape=(256))
features=Input(shape=(196,512))
decoder_input_states = [decoder_state_input_h, decoder_state_input_c]
decoder_inf_input=Input(shape=(1,len(voc)))
decoder_h, decoder_c, alfa = decoder([features,decoder_inf_input,decoder_input_states])
decoder_states = [decoder_h , decoder_c, alfa]
decoder_out = word_embedding(decoder_h)

# Create decoder inference model
decoder_model_inf = k.Model(inputs=[features,decoder_inf_input] + decoder_input_states, outputs=[decoder_out] + decoder_states )

with open("YOLO_LSTM_weights.pkl", "rb") as open_file:
    LSTM_list = pickle.load(open_file)
with open("YOLO_word_embedding_weights.pkl","rb") as open_file:
    embedding_list=pickle.load(open_file)
    
decoder.set_weights(LSTM_list)
word_embedding.set_weights(embedding_list)

In [8]:
def generate_words(inputs):
    feature = encoder_model_inf.predict(inputs)
    target_seq = np.zeros((1,1, len(voc)))
    target_seq[0,0, word_to_idx['\t']] = 1
    states_val=[np.zeros((1,256)),np.zeros((1,256))]
    decoder_out, decoder_h, decoder_c, _ = decoder_model_inf.predict(x=[feature,target_seq]+states_val)
    # Find out the next character from the Decoder output
    max_val_index = np.argmax(decoder_out[0,:])
    sampled_suffix_word = idx_to_word[max_val_index]
    words=[]
    words.append(sampled_suffix_word)
    i=0
    while (sampled_suffix_word!="\n" and i<30):
        target_seq = np.zeros((1,1,  len(voc)))
        target_seq[0,0, max_val_index] = 1
        
        # Initialize the decoder state to the states from last iteration
        states_val = [decoder_h[0], decoder_c[0]]

        # Get decoder output
        decoder_out, decoder_h, decoder_c, _ = decoder_model_inf.predict(x=[feature,target_seq] + states_val)
        # Get most probable next character and print it.
        max_val_index = np.argmax(decoder_out[0,:])
        sampled_suffix_word = idx_to_word[max_val_index]
        words.append(sampled_suffix_word)
        i+=1
    return words

In [9]:
def generate_caption(inputs):
    words=generate_words(inputs)
    words=list(dict.fromkeys(words))
    len_frase=len(words)
    boolean=False
    boolean2=False
    i_sug=0
    caption=""
    for i,word in enumerate(words):
        frase=""
        if word=="/n":
            continue
        elif word in Formas:
            if word!="irregular":
                frase=word[:-1]+"o"
            else:
                frase+=word
        elif word in Margenes:
            frase=word
        elif word in Orientaciones:
            frase="con orientación "+word
        elif word in Ecogenicidades:
            if word=="compleja":
                frase="con ecogenicidad compleja"
            elif word=="mixta":
                frase="con ecogenicidad mixta"
            else:
                frase=word[:-1]+"o"
        elif word in Posteriores:
            if word=="mixto":
                frase="con característica posterior mixta"
            elif word=="sin cambios":
                frase="sin cambios posteriores"
            else:
                frase="con "+word+ " posterior"
        elif word in Halos:
            if word=="no":
                frase="sin halo ecogénico"
            elif word=="sí":
                frase="con halo ecogénico"
        elif word in Sugestividades:
            frase="sugestivo de "+ word
            boolean=True
            i_sug=i
        elif word in BIRADS:
            if boolean:
                frase=" (BIRADS®"+word+")"
            else:
                i_sug=i
                if word=="2":
                    frase=" (BIRADS® "+word+")"
                if word=="3":
                    frase="probablemente benigno"+"(BIRADS® "+word+")"
                elif word=="4A":
                    frase="de baja sospecha"+"(BIRADS® "+word+")"
                elif word=="4B":
                    frase="de sospecha intermedia"+"(BIRADS® "+word+")"
                elif word=="4C":
                    frase="de alta sospecha"+"(BIRADS® "+word+")"
                elif word=="5":
                    frase="sugestivo de carcinoma"+"(BIRADS® "+word+")"
        else:
            if not boolean2:
                frase=". Resultado final aprendido por base de datos: "
                boolean2=True
            if word in Resultados:
                if word=="benign":
                    frase+="benigno"
                else:
                    frase+="maligno"
#            elif word in Tipo:
#                frase+=word
        if frase!="":
            if i==0:
                caption+="Nódulo "+frase
            elif word in Ecogenicidades:
                if word[:2]=="hi" or word[0]=="i":
                    caption+=" e "+frase
                else:
                    caption+=" y "+frase
            elif i==0:
                caption+="Nódulo "+frase
            elif frase[0]=="." or frase[:2]==" (":
                caption+=frase
            elif caption[-2:]==": ":
                caption+=frase
            elif i==len(frase)-1:
                caption+=", "+frase+"."
            else:
                caption+=", " +frase
    return caption
    

In [10]:
def results(imagen,imagen_dir="..\YOLO_para_describir\images"):
    imag_dir=os.path.join(imagen_dir,imagen)
    
    if not glob.glob(os.path.join("runs/detect/Yolo_test_lstm",imagen)):
        !python detect.py --weights YOLO_Caption448.pt --img 896 --conf 0.25 --source {imag_dir} --data data/YOLO.yaml --exist-ok --name Yolo_test_lstm --save-crop
    original=load_img(os.path.join("runs/detect/Yolo_test_lstm",imagen),color_mode="grayscale")
    
    nodulos=glob.glob(os.path.join("runs/detect/Yolo_test_lstm/crops/nodulo",imagen[:-4]+"*"))
    print("Número de nódulos: ",len(nodulos))
    display(original)
    for ima in nodulos:
        imag_or=load_img(ima,color_mode="grayscale")
        ancho,largo=imag_or.size
        imag=img_to_array(imag_or)/255.
        imag=tf.image.pad_to_bounding_box(
        imag, (tamaño-largo)//2,(tamaño-ancho)//2, 450, 450
        )
        imag=img_to_array(imag)
        words=generate_caption(imag.reshape(1,450,450,1))
#        visualize(imagen=imag,alfas=np.array(alfas[:,0]))
        display(imag_or)
        print(words)
    if not nodulos:
        print("Sin hallazgos. (BIRADS® 1)")

In [9]:
with contextlib.redirect_stdout(None):
    %cd yolov5

# Esto es una prueba

In [10]:
btn_upload = widgets.FileUpload()
out_pl=widgets.Output()
btn_run = widgets.Button(description='Descripción')
lbl_pred = widgets.Label()
def on_click_descripcion(change):
    img=Image.open(io.BytesIO(btn_upload.data[-1]))
    name=list(btn_upload.value.keys())[-1]
    img_dir=os.path.join("../Imagenes_detect",name)
    img.save(img_dir)
    out_pl.clear_output()
    with out_pl: 
        display(img)
        words=results(name,"../Imagenes_detect")
btn_run.on_click(on_click_descripcion)
VBox([widgets.Label('Selecciona una ecografía'), 
      btn_upload, btn_run, out_pl])

In [11]:
%load_ext watermark
# python, ipython, packages, and machine characteristics
%watermark -v -m -p wget,pandas,numpy,geopy,altair,vega,vega_datasets,watermark 

# date
print (" ")
%watermark -u -n -t -z 

Python implementation: CPython
Python version       : 3.7.11
IPython version      : 7.31.1

wget         : not installed
pandas       : 1.3.5
numpy        : 1.21.5
geopy        : not installed
altair       : not installed
vega         : not installed
vega_datasets: not installed
watermark    : 2.3.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 165 Stepping 2, GenuineIntel
CPU cores   : 16
Architecture: 64bit

 
Last updated: Tue Mar 29 2022 18:16:18Hora de verano romance

